К сожалению методами ИИ задачу решить пока не получилось. Перевод сделан простыми алгоритмами. Пока не все сделано, программа будет совершенствоваться.

In [40]:
import pandas as pd

import requests
import re
import string

import pymorphy2
lemmer = pymorphy2.MorphAnalyzer()

In [41]:
word_df = pd.read_csv('../lr3/witcher_words.csv')
phrase_df = pd.read_csv('../lr3/witcher_phrases.csv')  # Пока не используется.

In [44]:
def translate_text_to_elf(text: str) -> str:
    """Переводит текст на язык Старшей Речи."""

    text = text.replace('ё', 'е')
    for punc in string.punctuation:
        text = text.replace(punc, '')

    translated_words = []
    for word in text.lower().split():
        word = _word_to_lemm(word)
        translate = None
        try:
            translate = _try_to_find_russian_word_translate(word)
        except ValueError:
            for synonym in _synonyms_of_word(word):
                try:
                    translate = _try_to_find_russian_word_translate(word)
                    break
                except ValueError:
                    continue

        if translate is None:
            translate = transliterate_russian_to_latin(word)

        translated_words.append(translate)

    return ' '.join(translated_words)


def _word_to_lemm(word: str) -> str:
    """Переводит слово в начальную форму."""
    return lemmer.parse(word)[0].normal_form


def _try_to_find_russian_word_translate(word: str) -> str:
    """Ищет перевод русского слова в датасете."""
    result = word_df.loc[word_df['translation'] == word, 'text']
    if not result.empty:
        return result.values[0]
    else:
        raise ValueError


def _synonyms_of_word(word: str) -> str:
    """Выдаёт всевозможные синонимы заданного слова. Использует сетевой запрос."""
    html = requests.get(f'https://text.ru/synonym/{word}').text
    
    match = re.search(r'<meta name=\"description\" content=\"Синонимы к слову [^—]*:([^\"]+)\" />', html)
    if match is None:
        return []

    text = match.group(1)
    synonyms = [synonym.strip() for synonym in text.strip().split('—')]
    synonyms.remove('')

    return synonyms


def transliterate_russian_to_latin(russian_word: str):
    """Функция для латинизации русского слова."""
    translit_dict = {
        'а': 'a', 'б': 'b', 'в': 'v', 'г': 'g', 'д': 'd', 'е': 'e', 'ё': 'yo',
        'ж': 'zh', 'з': 'z', 'и': 'i', 'й': 'y', 'к': 'k', 'л': 'l', 'м': 'm',
        'н': 'n', 'о': 'o', 'п': 'p', 'р': 'r', 'с': 's', 'т': 't', 'у': 'u',
        'ф': 'f', 'х': 'kh', 'ц': 'ts', 'ч': 'ch', 'ш': 'sh', 'щ': 'shch', 'ъ': '',
        'ы': 'y', 'ь': '', 'э': 'e', 'ю': 'yu', 'я': 'ya'
    }

    latin_word = ''.join(translit_dict.get(char, char) for char in russian_word.lower())
    return latin_word

In [46]:
translate_text_to_elf('Здравствуй, Геральт! Моё имя - Аваллак\'х! Помоги мне найти ласточку!')

'cead geralt me aymm avallakkh pomoch aé nayti zireael'

In [47]:
translate_text_to_elf('Прощай, дочь. Мы еще увидимся.')

'va faill luned sinn vort uvidetsya'

In [48]:
translate_text_to_elf('Когда обед? Я хочу есть.')

'cathain obed aé wett ithte'

In [49]:
translate_text_to_elf('Почему этот курс такой сложный? Я не знаю.')

'pochemu etot kurs takoy slozhnyy aé neen ifit'